In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout,BatchNormalization

In [2]:
cifar10= tf.keras.datasets.cifar10

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

X_test = X_test / 255.0
X_train = X_train / 255.0

170498071/170498071 [==============================] - 3s 0us/step


In [3]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,Y_train)).batch(32).shuffle(1000)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test,Y_test)).batch(32)

In [4]:
X_train.shape

(50000, 32, 32, 3)

In [40]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(64,(2,2), activation="relu", padding="same")
    self.conv2 = Conv2D(64, (4,4), activation="relu", padding="same")
    self.conv3 = Conv2D(128, (2,2), activation="relu", padding="same")
    self.conv4 = Conv2D(32, (3,3), activation="relu", padding="same")
    self.b1 = BatchNormalization()
    self.b2 = BatchNormalization()
    self.b3 = BatchNormalization()
    self.b4 = BatchNormalization()
    self.m = MaxPooling2D(3,3)
    self.dropout1 = Dropout(0.3)
    self.dropout2 = Dropout(0.5)
    self.dropout3 = Dropout(0.4)
    self.dense1 = Dense(128, activation='relu')
    self.dense2 = Dense(10, activation='softmax')
    self.f = Flatten()

  def call(self, x):
    x = self.conv3(x)
    x = self.b1(x)
    x = self.conv2(x)
    x = self.dropout1(x)
    x = self.b2(x)
    x = self.conv1(x)
    x = self.b3(x)
    x = self.m(x)
    x = self.dropout2(x)
    x = self.b4(x)
    x = self.conv4(x)
    x = self.f(x)
    x = self.dense1(x)
    x = self.dropout3(x)
    x = self.dense2(x)
    return x

model = MyModel()

In [41]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [42]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.metrics.SparseCategoricalAccuracy()

In [43]:
@tf.function
def train(images,labels):
  with tf.GradientTape() as tape:
      pred = model(images)
      loss = loss_function(labels,pred)
  gradients = tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,pred)

In [44]:
@tf.function
def test(images, labels):
  pred = model(images)
  loss = loss_function(labels, pred)

  test_loss(loss)
  test_accuracy(labels, pred)

In [45]:
epochs = 5
for epoch in range(epochs):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()
  for images,labels in train_dataset:
    train(images,labels)


  for images,labels in test_dataset:
    test(images,labels)


  print("epoch:", epoch + 1,
          f"Train loss: {train_loss.result()}",
          f"Train accuracy: {train_accuracy.result()}",
          f"Test loss: {test_loss.result()}",
          f"test accuracy: {test_accuracy.result()}")

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


epoch: 1 Train loss: 1.294523000717163 Train accuracy: 0.5341200232505798 Test loss: 1.027211308479309 test accuracy: 0.6360999941825867
epoch: 2 Train loss: 0.8493925929069519 Train accuracy: 0.700160026550293 Test loss: 0.840398907661438 test accuracy: 0.7087000012397766
epoch: 3 Train loss: 0.6735411286354065 Train accuracy: 0.7634999752044678 Test loss: 0.7965424656867981 test accuracy: 0.7249000072479248
epoch: 4 Train loss: 0.5419658422470093 Train accuracy: 0.809440016746521 Test loss: 0.773866593837738 test accuracy: 0.7384999990463257
epoch: 5 Train loss: 0.4330258369445801 Train accuracy: 0.8463000059127808 Test loss: 0.8332168459892273 test accuracy: 0.7429999709129333
